In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import operator
import re
from collections import Counter
path='C:/Projects/Hackerearth/Problem 2/'
# path=''

In [3]:
data=pd.read_json(path+'train_data.json')
test_data=pd.read_json(path+'test_data.json')

In [4]:
train_data=data.transpose()
test=test_data.transpose()
train_data.shape,test.shape

((200000, 6), (100000, 5))

In [5]:
test=test.reset_index()
train_data=train_data.reset_index()

In [6]:
mappa={'neg':0,'pos':1}
train_data['segment']=train_data['segment'].map(mappa)

In [7]:
all_data=pd.concat([train_data,test],axis=0)
all_data=all_data.reset_index(drop=True)
all_data.columns=[u'cities', u'dow', u'genres', u'user_id', u'segment', u'titles', u'tod']

In [8]:
all_data['no_of_Cities']=all_data['cities'].apply(lambda x:len(x.split(',')) )
all_data['no_of_dow']=all_data['dow'].apply(lambda x:len(x.split(',')) )
all_data['no_of_tod']=all_data['tod'].apply(lambda x:len(x.split(',')) )
all_data['no_of_genre']=all_data['genres'].apply(lambda x:len(x.split(',')) )
all_data['no_of_title']=all_data['titles'].apply(lambda x:len(x.split(',')) )
all_data['total_time_spent']=all_data['cities'].apply(lambda x:sum([int(i.split(':')[1] )for i in x.split(',')]))
all_data['dow']=all_data['dow'].apply(lambda x:[[i.split(':')[0],int(i.split(':')[1])] for i in x.split(',')])
all_data['tod']=all_data['tod'].apply(lambda x:[[i.split(':')[0],int(i.split(':')[1])] for i in x.split(',')])
all_data['genres']=all_data['genres'].apply(lambda x:[[i.split(':')[0],int(i.split(':')[1])] for i in x.split(',')])

In [9]:
def value_dow(x,a):
    try:
        return [i[1] for i in x if i[0]==a][0]
    except:
        return 0
    
def greaterthan(x):
    if x>= 1:
        return 1
    else:
        return 0    

genre_list=set([item[0] for kk in all_data['genres'] for item in kk])

In [10]:
list_of_var=[]
for kk in range(1,8):
    list_of_var.append('dow_'+str(kk))
    all_data['dow_'+str(kk)]=all_data['dow'].apply(lambda x:value_dow(x,str(kk)))
    all_data['dow_'+str(kk)+'_percent']=all_data['dow_'+str(kk)]/all_data['total_time_spent']
    
for kk in range(0,24):
    list_of_var.append('tod_'+str(kk))
    all_data['tod_'+str(kk)]=all_data['tod'].apply(lambda x:value_dow(x,str(kk)))
    all_data['tod_'+str(kk)+'_percent']=all_data['tod_'+str(kk)]/all_data['total_time_spent']
    
for ee in genre_list:
    list_of_var.append('genres_'+str(ee))
    all_data['genres_'+str(ee)]=all_data['genres'].apply(lambda x:value_dow(x,str(ee)))
    all_data['genres_'+str(ee)+'_percent']=all_data['genres_'+str(ee)]/all_data['total_time_spent']

In [11]:
for cola in list_of_var:
    all_data[cola+'_present']=all_data[cola].apply(lambda x:greaterthan(x) )

In [12]:
all_data['average_per_day']= all_data['total_time_spent']/all_data['no_of_dow']
all_data['average_per_city']= all_data['total_time_spent']/all_data['no_of_Cities']
all_data['average_per_time']= all_data['total_time_spent']/all_data['no_of_tod']
all_data['average_per_title']= all_data['total_time_spent']/all_data['no_of_title']
all_data['average_per_genre']= all_data['total_time_spent']/all_data['no_of_genre']
all_data['title_per_genre']=all_data['no_of_title']/all_data['no_of_genre']
all_data['sum_of_tamasha']=all_data[['no_of_title','no_of_dow','no_of_Cities','no_of_tod','no_of_genre']].sum(axis=1)

In [13]:
title_all=[]
for ffff in all_data['titles']:
    kkkk=re.sub('\d*','',ffff).replace(':','').split(',')
    title_all.append(kkkk)
    
title_list=[topa for top in title_all for topa in top]

count_of_tile=Counter(title_list)

dica_title={}
for tita in  count_of_tile:
    dica_title[tita]=count_of_tile[tita]
    
sorted_x=sorted(dica_title.items(), key=operator.itemgetter(1),reverse=True)

title_cols=[ j[0] for j in  sorted_x if j[1] > 1000]

In [14]:
all_data['titles']=title_all

In [15]:
def title_tally(y,x):
    if y in x:
        return 1
    else:
        return 0

In [16]:
for x,claa in enumerate(title_cols):
    all_data['title_00'+str(x) ] = all_data['titles'].apply(lambda x: title_tally(claa,x) )

In [17]:
exper_data=all_data[:200000].copy()
del exper_data['cities']
del exper_data['dow']
del exper_data['tod']
del exper_data['genres']
del exper_data['titles']
del exper_data['user_id']

In [18]:
exper_data.shape

(200000, 752)

In [19]:
exper_data.to_csv(path+'new_trains.csv',index=False)

In [20]:
del data

In [21]:
del exper_data

In [22]:
naya_data=all_data.copy()
del naya_data['cities']
del naya_data['dow']
del naya_data['tod']
del naya_data['genres']
del naya_data['titles']
del naya_data['user_id']
del naya_data['segment']

In [23]:
naya_data.shape

(300000, 751)

In [24]:
naya_data[200000:].to_csv(path+'new_test.csv',index=False)

In [ ]:
naya_data.shape,exper_data.shape

In [ ]:
target=train_data['segment']

In [ ]:
model=xgb.XGBClassifier()

In [ ]:
model.fit(naya_data[:200000], target)

In [ ]:
pred=model.predict_proba(naya_data[200000:])

In [ ]:
pred[::,1]

In [ ]:
sub=pd.concat([test['index'],pd.DataFrame(pred[::,1])],axis=1)
sub.columns=['ID','segment']
sub.to_csv(path+'ghrprob2xgb.csv',index=False)

### RF

In [ ]:
from sklearn import ensemble

In [ ]:
rf1=ensemble.RandomForestClassifier(n_estimators=100)
rf1.fit(naya_data[:200000], target)
rfpred=rf1.predict_proba(naya_data[200000:])

In [ ]:
sub2=pd.concat([test['index'],pd.DataFrame(rfpred[::,1])],axis=1)
sub2.columns=['ID','segment']
sub2.to_csv('prob2rf.csv',index=False)

### mean

In [ ]:
np.corrcoef(sub['segment'],sub2['segment'])

In [ ]:
sub3=(sub['segment']+sub2['segment'])/2

In [ ]:
sub4=pd.concat([test['index'],pd.DataFrame(sub3)],axis=1)
sub4.columns=['ID','segment']
sub4.to_csv('prob2mean.csv',index=False)

### weighted

In [ ]:
sub5=(.75*(sub['segment'])+.25*(sub2['segment']))

In [ ]:
sub6=pd.concat([test['index'],pd.DataFrame(sub5)],axis=1)
sub6.columns=['ID','segment']
sub6.to_csv('prob2weighted.csv',index=False)